In [1]:
import os
import numpy as np 
import pandas as pd
from glob import glob
import cv2
import random
import matplotlib.pyplot as plt
import joblib
from tensorflow.keras.models import load_model

In [2]:
dtype_dict = {
    'code': str
}

In [3]:
predict_file = pd.read_csv(r'C:\Users\USER\Paper\with index_V3\Prediction_path.csv', dtype=dtype_dict).iloc[:,1:]
predict_path = r'C:\Users\USER\Paper\with index_V3\\'
predict_file['File Paths'] = predict_file['File Paths'].apply(lambda path:os.path.join(predict_path, path))
df_drop_predict = predict_file.drop_duplicates('code',keep='first').reset_index(drop=True)

In [4]:
model = load_model(r'C:\Users\USER\Paper\with index_V3\model_V2.h5')

In [5]:
prediction = pd.DataFrame()

img_list_predict = []

for i in range(len(predict_file)):
    img_predict = cv2.imread(predict_file['File Paths'].loc[i])
    img_list_predict.append(img_predict)
X_test = np.concatenate(img_list_predict, axis = 0)

X_test = X_test.reshape(len(predict_file), 170,340,3)/255
y_prob = model.predict(X_test)
y_prob_flat = np.ravel(y_prob)
y_prob_df = pd.DataFrame({'predict_rate':y_prob_flat})
y_prob_df['Decision'] = y_prob_df['predict_rate'].apply(lambda value:'Sell' if value<=0.0003 else ('Buy' if value >= 0.999 else 'No Action'))

predict_df = pd.concat([predict_file['code'],
                        predict_file['Name'],
                        predict_file['Date'],
                        y_prob_df['Decision'],
                        y_prob_df['predict_rate'],
                        predict_file['Close'],
                        predict_file['Volume']],
                       axis=1
                      )

In [6]:
dt = '0408'
predict_df.to_excel('prediction' + dt + '.xlsx')

In [7]:
predict_df

,code,Name,Date,Decision,predict_rate,Close,Volume
0,0050,元大台灣50,2025-04-07,No Action,0.564424,158.40,4685331
1,0051,元大中型100,2025-04-07,No Action,0.565239,65.70,12182
2,0052,富邦科技,2025-04-07,No Action,0.605168,156.05,60859
3,0053,元大電子,2025-04-07,No Action,0.590052,83.85,709
4,0055,元大MSCI金融,2025-04-07,No Action,0.694789,25.44,12039
...,...,...,...,...,...,...,...
3080,Y8886,未含金電,2025-04-07,No Action,0.989022,12819.68,26243115
3081,Y8888,不含金融,2025-04-07,No Action,0.815416,16639.71,92291972
3082,Y9997,報酬指數,2025-04-02,No Action,0.632108,47206.32,240146859
3083,Y9998,TEJ 大盤,2025-04-07,No Action,0.893243,19232.35,106762707
